<a href="https://colab.research.google.com/github/whataLIN/Bdata23th/blob/main/whataLIN/ch09_DL_14_KoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* https://github.com/treblenalto/korean-speech-emotion-recognition
* https://hoit1302.tistory.com/159

* 구글에서 2018년에 공개한 BERT는 등장과 동시에 수많은 NLP 태스크에서 최고 성능을 보여주면서 NLP의 한 획을 그은 모델로 평가받고 있다. 양방향성을 지향하고 있기 때문이다.(B: bidirection) BERT 모델은 문맥 특성을 활용하고 있고, 대용량 말뭉치로 사전 학습이 이미 진행되어 언어에 대한 이해도도 높다.  하지만 BERT는 한국어에 대해서 영어보다 정확도가 떨어진다고 한다. 

* 오늘 기술해볼 KoBERT 모델은 SKTBrain에서 공개했는데, 한국어 위키 5백만 문장과 한국어 뉴스 2천만 문장을 학습한 모델이다. 자신의 사용 목적에 따라 파인튜닝이 가능하기 때문에 output layer만을 추가로 달아주면 원하는 결과를 출력해낼 수 있다. 많은 BERT 모델 중에서도 KoBERT를 사용한 이유는 "한국어"에 대해 많은 사전 학습이 이루어져 있고, 감정을 분석할 때, 긍정과 부정만으로 분류하는 것이 아닌 다중 분류가 가능한 것이 강점이기 때문이다.

* 이 실습에서 하는 작업은 바로 파인 튜닝(Fine-tuning)에 해당한다. 다른 작업에 대해서 파라미터 재조정을 위한 추가 훈련 과정을 거치는 것을 말한다. 예시를 들어보면, 우리가 하고 싶은 작업이 우울증 경향 문헌 분류라고 하였을 때, 이미 위키피디아 등으로 사전 학습된 BERT 위에 분류를 위한 신경망을 한 층 추가하는 것이다. 이미 BERT가 언어 모델 사전 학습 과정에서 얻은 지식을 활용할 수 있으므로 우울증 경향 문헌 분류에서 보다 더 좋은 성능을 얻을 수 있다.

* https://github.com/SKTBrain/KoBERT

# 의존성

In [ ]:
!pip install gluonnlp pandas tqdm -q
!pip install mxnet -q
!pip install sentencepiece==0.1.91 -q
!pip install transformers==4.8.2 -q
!pip install torch -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 KB 11.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 59.4 MB/s eta 

# KoBERT 모델

In [ ]:
# https://huggingface.co/skt/kobert-base-v1
# 토크나이저
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf' -q
# 모델
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master -q

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 KB 17.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu11

In [ ]:
# BERT는 이미 누군가가 학습해둔 모델을 사용한다(pre-trained model)는 것을 뜻한다.
# 따라서 사용하는 model과 tokenizer는 항상 mapping 관계여야 한다.
# 예를 들어서 U 팀이 개발한 BERT를 사용하는데, V팀이 개발한 BERT의 tokenizer를 사용하면 model은 텍스트를 이해할 수 없다.
# U팀의 BERT의 토크나이저는 '우리'라는 단어를 23번으로 int encoding하는 반면에,
# V라는 BERT의 tokenizer는 '우리'라는 단어를 103번으로 int encoding해 단어와 mapping 되는 정보 자체가 달라지기 때문이다.
from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab = get_pytorch_kobert_model()
# https://huggingface.co/skt/kobert-base-v1

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# 필요한 라이브러리

In [ ]:
# 사전 학습된 BERT를 사용할 때는 transformers라는 패키지를 자주 사용한다.
# 또한, 학습시간을 줄이기 위해, GPU를 사용했다.
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp # http://freesearch.pe.kr/archives/4963
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = "cuda" if torch.cuda.is_available() else "cpu"

# 데이터셋 불러오기

In [ ]:
data = pd.read_csv('https://github.com/treblenalto/korean-speech-emotion-recognition/raw/main/data/most_common.csv')

In [ ]:
len(data)

10004

In [ ]:
data.head()

,speech,emotion
0,개를 예쁘다고 사놓고 끝까지 키우지도 않고 버리는 사람들이 엄청 많아졌대.,Angry
1,지금도 그대로 있어. 치우는 사람이 없어.,Disgust
2,맞아. 무기력증인 것 같아. 한동안 정말 바빴었거든.,Sadness
3,오늘이 발표날인데 연락이 없더라고. 그래서 알아봤더니 명단에 내 이름이 없대.,Sadness
4,그치. 개 키우는 사람이 늘어나니까 그만큼 버리는 사람도 늘어나는 거야!,Angry


In [ ]:
data.emotion.value_counts()

Sadness      6009
Angry        2194
Fear          929
Neutral       423
Disgust       272
Happiness     164
Surprise       13
Name: emotion, dtype: int64

In [ ]:
data['emotion'] = data['emotion'].map({
    'Fear': '0', # 공포
    'Surprise': '1', # 놀람
    'Angry': '2', # 분노
    'Sadness': '3', # 슬픔
    'Neutral': '4', # 중립
    'Happiness': '5', # 기쁨
    'Disgust': '6', # 혐오
})

#  7개의 감정 class를 0~6 숫자에 대응시켜 data_list에 담아준다.
# data_list = []
# for ques, label in zip(data['speech'], data['emotion'])  :
#     data = []   
#     data.append(ques)
#     data.append(str(label))

#     data_list.append(data)

In [ ]:
data.head()

,speech,emotion
0,개를 예쁘다고 사놓고 끝까지 키우지도 않고 버리는 사람들이 엄청 많아졌대.,2
1,지금도 그대로 있어. 치우는 사람이 없어.,6
2,맞아. 무기력증인 것 같아. 한동안 정말 바빴었거든.,3
3,오늘이 발표날인데 연락이 없더라고. 그래서 알아봤더니 명단에 내 이름이 없대.,3
4,그치. 개 키우는 사람이 늘어나니까 그만큼 버리는 사람도 늘어나는 거야!,2


# 입력 데이터셋을 토큰화

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
        # gluonnlp
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         
    def __len__(self):
        return (len(self.labels))
# 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding 등을 해주는 코드이다.

# 훈련 준비

In [ ]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5  
# num_epochs = 10  
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5 # 0.00005

In [ ]:
data['emotion'].value_counts()

3    6009
2    2194
0     929
4     423
6     272
5     164
1      13
Name: emotion, dtype: int64

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

# 7개인데 7개가 균형잡히게 배치 -> 계층화하여 훈련셋/테스트셋 분리
dataset_train, dataset_test = train_test_split(data.values, test_size=0.2,
                                               shuffle=True, random_state=26,
                                               stratify=data['emotion'])

In [ ]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab,  max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

# KoBERT 모델 구현하기

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(),
                              attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 모델 훈련

In [ ]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

  0%|          | 0/126 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9849225282669067 train acc 0.125
epoch 1 train acc 0.5771329365079365


  0%|          | 0/32 [00:00<?, ?it/s]

epoch 1 test acc 0.8099149816176471


  0%|          | 0/126 [00:00<?, ?it/s]

...

epoch 4 test acc 0.8771829044117647


  0%|          | 0/126 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.15447251498699188 train acc 0.953125
epoch 5 train acc 0.9408482142857143


  0%|          | 0/32 [00:00<?, ?it/s]

epoch 5 test acc 0.8849954044117647


# 직접 만든 새로운 문장으로 테스트

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()
            print(logits)
            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 감기 걸려서 아파


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[ 0.19896086 -2.5616686   0.6125719   5.5843534  -2.0924897  -2.0130036
 -1.8160303 ]
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 오늘 밥을 너무 많이 먹었어
[ 0.26692873 -2.810272    1.4958541   4.9753976  -2.0069745  -2.4047265
 -1.9488509 ]
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 너무 기쁘다
[-0.21798818 -2.2939715  -0.6463571   5.735091   -1.5352685  -0.918401
 -1.7407556 ]
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 그치!
[ 0.60294    -1.4745481   3.2523615   0.43040454 -1.4349321  -2.636874
  0.5542462 ]
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : O
[ 0.10209588 -2.6349223   2.0957873   2.6960502  -0.6477472  -2.4936473
 -0.79069364]
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 0
